# Window and custom features

When forecasting time series data, it may be useful to consider additional characteristics beyond just the lagged values. For example, the moving average of the previous *n* values may help to capture the trend in the series. The `window_features` argument allows the inclusion of additional predictors created with the previous values of the series.

## Libraries and data

In [1]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.datasets import fetch_dataset
from skforecast.recursive import ForecasterRecursive
from skforecast.preprocessing import RollingFeatures
from lightgbm import LGBMRegressor

In [2]:
# Data
# ==============================================================================
data = fetch_dataset(name="h2o", raw=False)
data.index.name = 'datetime'
data = data.rename(columns={'x': 'y'})
y = data['y']
y

h2o
---
Monthly expenditure ($AUD) on corticosteroid drugs that the Australian health
system had between 1991 and 2008.
Hyndman R (2023). fpp3: Data for Forecasting: Principles and Practice(3rd
Edition). http://pkg.robjhyndman.com/fpp3package/,https://github.com/robjhyndman
/fpp3package, http://OTexts.com/fpp3.
Shape of the dataset: (204, 1)


datetime
1991-07-01    0.429795
1991-08-01    0.400906
1991-09-01    0.432159
1991-10-01    0.492543
1991-11-01    0.502369
                ...   
2008-02-01    0.761822
2008-03-01    0.649435
2008-04-01    0.827887
2008-05-01    0.816255
2008-06-01    0.762137
Freq: MS, Name: y, Length: 204, dtype: float64

## RollingFeatures

The <code>RollingFeatures</code> class available is skforecast allows the creation of some of the most commonly used predictors:

+ 'mean': the mean of the previous *n* values.
+ 'std': the standard deviation of the previous *n* values.
+ 'min': the minimum of the previous *n* values.
+ 'max': the maximum of the previous *n* values.
+ 'sum': the sum of the previous *n* values.
+ 'median': the median of the previous *n* values.
+ 'ratio_min_max': the ratio between the minimum and maximum of the previous *n* values.
+ 'coef_variation': the coefficient of variation of the previous *n* values.

The user can specify these predictors by passing a list of strings to the `stats` argument. The user can also specify the window size for each of these predictors by passing a `list` of integers to the `window_size` argument, if the value is the same for all the predictors, the user can pass a single `integer`.

The following example demonstrates how to use the <code>RollingFeatures</code> class to include rolling statistics (mean, minimum, and maximum values). Here, the rolling mean is computed with a window size of 20, while the minimum and maximum values use a window size of 10.

In [ ]:
# Window features
# ==============================================================================
window_features = RollingFeatures(
                      stats        = ['mean', 'min', 'max'],
                      window_sizes = [20, 10, 10]
                  )

In [ ]:
# Create and fit forecaster
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=123, n_jobs=-1, verbose=-1),
                 lags            = 3,
                 window_features = window_features
             )

forecaster.fit(y=y)
forecaster

In [5]:
# Predict
# ==============================================================================
steps = 36
predictions = forecaster.predict(steps=steps)
predictions.head(3)

2008-07-01    0.880138
2008-08-01    1.020947
2008-09-01    1.061728
Freq: MS, Name: pred, dtype: float64

The window size needed by this Forecaster is the maximum of the window sizes between the lagged values and the rolling features. In this case, this value is 20.

In [ ]:
# Predict
# ==============================================================================
print("Window size lags            : ", forecaster.max_lag)
print("Window size window features : ", forecaster.max_size_window_features)
print("Window size Forecaster      : ", forecaster.window_size)

Additional arguments can be passed to the `RollingFeatures`:

+ `features_names`: By default, feature names follow the pattern `roll_<stat>_<window_size>`. For instance, a rolling mean with a window size of 20 is named `roll_mean_20`. Users can also assign custom names to each feature using a `list` of strings.

+ `min_periods`: allows specifying the minimum number of observations required to compute the statistics during the training matrix generation (same as the `min_periods` argument of [pandas rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)). It can be a single integer or a list of integers, one for each statistic.

+ `fill_na`: define the strategy for handling missing values during the training matrix generation. Available methods are: `'mean'`, `'median'`, `'ffill'`, `'bfill'`, or a `float` value.

By inspecting the training matrices, it is possible to check that the rolling features have been correctly included.

In [6]:
# Training matrices used internally to fit the regressor
# ==============================================================================
X_train, y_train = forecaster.create_train_X_y(y=y)
X_train

,lag_1,lag_2,lag_3,roll_mean_20,roll_min_10,roll_max_10
datetime,,,,,,
1993-03-01,0.387554,0.751503,0.771258,0.496401,0.361801,0.771258
1993-04-01,0.427283,0.387554,0.751503,0.496275,0.387554,0.771258
1993-05-01,0.413890,0.427283,0.387554,0.496924,0.387554,0.771258
1993-06-01,0.428859,0.413890,0.427283,0.496759,0.387554,0.771258
1993-07-01,0.470126,0.428859,0.413890,0.495638,0.387554,0.771258
...,...,...,...,...,...,...
2008-02-01,1.219941,1.176589,1.163534,0.980390,0.561760,1.219941
2008-03-01,0.761822,1.219941,1.176589,0.978582,0.745258,1.219941
2008-04-01,0.649435,0.761822,1.219941,0.966838,0.649435,1.219941


It is also possible to use the <code>RollingFeatures</code> class outside the forecaster to gain a deeper insight into its behaviour. The `transform` method computes the rolling features for a given numpy array, which is assumed to contain as many past observations as the maximum window size required to compute the features. The output is a numpy array with the rolling features.

In [7]:
# Create rolling features from a given array
# ==============================================================================
x = np.arange(20)
window_features.transform(X=x)

array([ 9.5, 10. , 19. ])

The `transform_batch` method is designed to transform a whole pandas Series from which multiple rolling windows can be extracted. The output is a pandas DataFrame with the rolling features.

In [8]:
# Create rolling features from a pandas series
# ==============================================================================
window_features.transform_batch(y).head(3)

,roll_mean_20,roll_min_10,roll_max_10
datetime,,,
1993-03-01,0.496401,0.361801,0.771258
1993-04-01,0.496275,0.387554,0.771258
1993-05-01,0.496924,0.387554,0.771258


The reason for these two different data transformation methods is that the first is used during prediction, where the forecaster only has access to the last window of the series. In contrast, the second is used during training, where the forecaster has access to the entire series.

## Create your custom window features

<code>RollingFeatures</code> is very useful for including some of the most commonly used predictors.  However, users may need to include additional predictors that are not provided by this class. In such cases, users can create their own custom class to compute the desired features and include them in the forecaster.

The custom class must have these 2 methods:

+ `transform_batch`: method to compute the features in batch from a pandas Series. This method will be used to compute the features during the training process. It must return a pandas DataFrame containing the rolling features.

+ `transform`: method to compute the features from a numpy array. This method will be used to compute the features during the prediction process. It must return a numpy array containing the computed statistics.

and these 2 attributes:

+ `window_sizes`: size of the rolling window required to compute the features. It must be a list of integers.

+ `features_names`: list with the names of the output features. It must be a list of strings.

The follwing example shows how to create a custom class to include the rolling skewness and kurtosis with a window size of 20.

<div class="admonition note" name="html-admonition" style="background: rgba(0,191,191,.1); padding-top: 0px; padding-bottom: 6px; border-radius: 8px; border-left: 8px solid #00bfa5; border-color: #00bfa5; padding-left: 10px; padding-right: 10px;">

<p class="title">
    <i style="font-size: 18px; color:#00bfa5;"></i>
    <b style="color: #00bfa5;">&#128161 Tip</b>
</p>

If you have any doubt when creating your custom class, you can check the source code of the <code>RollingFeatures</code> class available in the <a href="../api/preprocessing.html#skforecast.preprocessing.preprocessing.RollingFeatures">API Reference</a>.

</div>

In [ ]:
# Custom class to create rolling skewness features
# ==============================================================================
from scipy.stats import skew


class RollingSkewness():
    """
    Custom class to create rolling skewness features.
    """

    def __init__(self, window_sizes, features_names: list = 'rolling_skewness'):
        
        if not isinstance(window_sizes, list):
            window_sizes = [window_sizes]
        self.window_sizes = window_sizes
        self.features_names = features_names

    def transform_batch(self, X: pd.Series) -> pd.DataFrame:
        
        rolling_obj = X.rolling(window=self.window_sizes[0], center=False, closed='left')
        rolling_skewness = rolling_obj.skew()
        rolling_skewness = pd.DataFrame({
                               self.features_names: rolling_skewness
                           }).dropna()

        return rolling_skewness

    def transform(self, X: np.ndarray) -> np.ndarray:
        
        X = X[~np.isnan(X)]
        if len(X) > 0:
            rolling_skewness = np.array([skew(X, bias=False)])
        else:
            rolling_skewness = np.nan
        
        return rolling_skewness

In [ ]:
# Transform batch
# ==============================================================================
window_features = RollingSkewness(window_sizes=3)
window_features.transform_batch(y)

,rolling_skewness
datetime,
1991-10-01,-1.696160
1991-11-01,0.897261
1991-12-01,-1.602797
1992-01-01,1.681518
1992-02-01,-0.778727
...,...
2008-02-01,1.359033
2008-03-01,-1.674974
2008-04-01,1.466482


In [ ]:
# Transform
# ==============================================================================
window_features.transform(X=np.array([6, 12, 8]))

array([0.93521953])

In [ ]:
# Forecaster with custom rolling features
# ==============================================================================
forecaster = ForecasterRecursive(
                 regressor       = LGBMRegressor(random_state=123, n_jobs=-1, verbose=-1),
                 lags            = 3,
                 window_features = window_features,
             )

forecaster.fit(y=y)
forecaster.predict(steps=5)